In [1]:
import os
from pprint import pprint
from sqlalchemy import insert, select

import settings # type: ignore
from camtrappy.db.schema import Video, Location, Project
from camtrappy.db.testdb import initialize_new_project, Session
from camtrappy.io import video, parsing


# Set data Dir & Data Project (file parser)
DATA_DIR = settings.data_dir_real
DATA_PROJECT = parsing.Project(name='Test', projectfolder=None, datafolder=DATA_DIR, restrict_to=[])

# Create new db project with parsed data
initialize_new_project(DATA_PROJECT)

# See how the data is structured
pprint(DATA_PROJECT.data(full_path=True))
pprint(DATA_PROJECT.data(full_path=False))
pprint(DATA_PROJECT.data_flat())
print(DATA_PROJECT.locations())

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
2021-07-18 19:09:20,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-18 19:09:20,333 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("videos")
2021-07-18 19:09:20,333 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 19:09:20,336 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("videos")
2021-07-18 19:09:20,336 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 19:09:20,337 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-07-18 19:09:20,338 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 19:09:20,339 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-07-18 19:09:20,340 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 19:09:20,341 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2021-07-18 19:09:20

In [4]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)
#     session.add_all([objects])
#     result = session.query(Project).filter(Project.name == 'Test').scalar() # scalar is good if only one match is possible due to database constraints